## Transformer 论文解读

&emsp;&emsp;之前学习Transformer的时候，主要注意的是attention的机制和论文的模型结构，并且对tensorflow的代码大致了解过，有些细节没有详细研究过，论文有些trick也没有注意到，现在重新整理一下思路，并结合代码进行解读。

### 1. 本次整理发现的细节

1. residual dropout:apply dropout to the output of each sub-layer,before it is added to the sub-layer input and normalized, In addition,we apply dropout to the sums of the embeddings and the positional encodings in both the encoder and decoder stacks.
2. label smoothing: 
3. embedding:论文中说we share tghe weight matrix between the two embedding layers,那是因为它把encoder的预料和decoder的预料合并在一起了，我们实现的时候可以跳过这个trick。

### 2. Transformer 架构

&emsp;&emsp;Transformer架构也使用的是encoder-decoder结构，如下图所示：

<img src="./pic/transformer.png" width = 30% height = 30% div align=center />

- 输入序列经过word embedding 和 positional embedding相加后，输入到encoder
- 输出序列经过word embedding 和 positional embedding相加后，输入到decoder
- decoder输出的结果经过 linear ，softmax后，计算下一个预测的单词

#### 2.1 Encoder

&emsp;&emsp;encoder由6层相同的层组成，每层分别由两部分组成：

- 第一部分是multi-head attention
- 第二部分是position-wise feed-forward network

&emsp;&emsp;这两个部分，都有一个residual connection，然后连接一个 layer normalization

#### 2.2 Decoder

&emsp;&emsp;decoder和encoder类似，每一层包含以下三个部分:

- 1.multi-head self-attention 
- 2.multi-head context-attention
- 3.position-wise feed-forward network 

&emsp;&emsp;这三个部分，都有一个residual connection，然后连接一个 layer normalization

#### 2.3 Attention

&emsp;&emsp;Attention机制就是对于某个时刻的输出y,它在输入x上各个部分的注意力，就是权重。

<img src="./pic/attention.jpg" width=30% height=30% div align=center/>

&emsp;&emsp;对于输入序列的hidden state $h_i$和输出序列的hidden state $s_t$，计算attention_score方法有三种，在transformer中采用的是乘性注意力机制，即使两个hidden state进行点积运算。

##### 2.3.1 self-Attention

&emsp;&emsp;self-attention就是输出序列就是输入序列，就是在word embedding的时候，不光考虑自己这个单词，也需要考虑这句话中其他的单词，这里采用乘性注意力机制的一个好处就是，一个向量点积自己一定是最大的，因此self-attention机制计算的得分总是关注自己的得分最大，也符合我们单词编码的自觉，一个token的编码肯定是和自己最相关的。

##### 2.3.2 context-Attention

&emsp;&emsp;context-attention 就是在decode的t时刻，获取该时刻在encoder的context 向量，此时的attention是decoder和encoder各个时刻的token的attention得分。

##### 2.3.3 Scaled dot-product Attention

&emsp;&emsp;论文中的计算公式如下：

<img src="./pic/scaled_attention.jpg" width=40% height=40% div align=center />

&emsp;&emsp;结合我的理解对query,key,value进行简单的讲解，目前存在一个字典，dict = {key:value},key和value都是一个向量，目前来一个查询query,我们想知道query对应的value是多少，但是在dict中没有query,因此最简单的做法就是将字典中的value进行average作为query对应的value,但是这样对任意一个query,得到的value都是一样的，因此这种处理方式不好，我们希望可以通过计算query和key之间的sim得分，来得到一个dict中所有value的加权权重，这样给query的value具有一定的信服力。

&emsp;&emsp;那么结合transformer，具体说明以下query,key,value是什么？

- 1.在encoder的self-attention中，query,key,value都来自上一层encoder的输出，对于第一层encoder，它们就是word embedding + positional embedding
- 2.在decoder的self-attention中，query,key,value都来自上一层decoder的输出，对于第一层decoder，它们就是word embedding + positional embedding，但是对于decoder,不希望它能获得下一个时刻的信息，因此需要进行sequence masking
- 3.在encoder-decoder attetnion中，query来自decoder的上一层的输出，key,value来自encoder的输出。

##### 2.3.4 Multi-head Scaled dot-product Attention

&emsp;&emsp;论文提到，将query,key,value 通过一个线性映射之后，分成h份，然后对每一份进行scaled dot-product attention，然后把各个部分的结果合并起来，再次经过线性映射，得到最终的输出。

&emsp;&emsp;论文中的说明如下：

<img src="./pic/multi-head.jpg" width=80% height=80% div align=center />

&emsp;&emsp;scaled dot-product 和 multi-head scaled dot-product的图解

<img src="./pic/attention_crontast.jpg" width=80% height=80% div align=center />

#### 2.4 Residual connection

&emsp;&emsp;就是残差连接，假设网络中某个层对输入x作用后的输出是F(x),那么增加residual connection之后就是 x + F(x)。

&emsp;&emsp;残差连接好处就是在反向传播的过程中，因为增加了常数项1，所以不会造成梯度消失。

#### 2.5 layer normalization

&emsp;&emsp;layer normalization是在每一个样本上计算均值和方差，公式如下：

$$LN(x_i) = \alpha * \frac{x_i - u_L}{\sqrt{\delta_l^2 + e}} + \beta$$

&emsp;&emsp;其中 $\alpha$ 和 $\beta$ 是可学习的参数。

#### 2.6 mask

&emsp;&emsp;mask就是掩码的意思，就是对某些值进行掩盖，使其不产生效果，在transformer中涉及到两种mask，padding mask 和 sequence mask。

##### 2.6.1 padding mask

&emsp;&emsp;每个批次的输入序列长度不一样，但是为了保证输入长度一样，对长度不够的句子用padding token进行填补，因为这些填补的位置，其实是没有什么意义的，所以attention机制不应该把注意力放在这些位置上，因此需要进行处理。

&emsp;&emsp;具体做法就是在这些padding位置上加一个非常大的负数，经过softmax后，这些位置的概率就会接近0。

##### 2.6.2 sequence mask

&emsp;&emsp;sequence mask 是为了使得decoder不能看见未来的信息，即对于一个序列，在t时刻的时候，解码的输出只能依赖于t时刻之前的输出，因此需要将t时刻之后的信息隐藏起来。也就是在t时刻，只能看到1 ... t-1时刻的信息。

#### 2.7 positional encoding

&emsp;&emsp;目前的transformer架构缺点东西，它对序列的顺序没有约束，没有positional encoding，它就是词袋模型，可能一句话里面的token 都预测正确，但是顺序预测不正确，论文中提出了positional encoding。

&emsp;&emsp;论文采用正余弦函数进行编码，公式如下

$$PE(pos,2i) = sin(pos/10000^{2i/d_model})$$

$$PE(pos,2i+1) = cos(pos/10000^{2i/d_model})$$

&emsp;&emsp;其中pos是指词语在序列中的位置，i是维度，上边这个公式不是很直观，其实就是将维度i分为奇数和偶数。假设$d_model$=4，那么可以分成两个pair:(0,1),(2,3),其中一个pair中的周期是相同的。

&emsp;&emsp;上面的位置对应绝对位置编码，但是词语的相对位置也非常重要

$$sin(a + b) = sin(a)cos(b) + cos(a)sin(b)$$

$$cos(a + b) = cos(a)cos(b) - sin(a)sin(b)$$

&emsp;&emsp;上面的公式说明，对于词汇之间的位置偏移k，PE(pos+k)可以表示为PE(pos)和PE(k)的组合形式。

#### 2.8 word embedding

&emsp;&emsp;in the embedding layers,multiply those weights by $\sqrt{d_model}$

#### 2.9 position-wise feed-forward network

&emsp;&emsp;就是一个全连接层，包含两个线性变换和一个非线性函数(relu),公式如下：

$$FFN(x) = max(0,xW_1 + b_1)W_2 + b_2$$

### 3. pytorch 实现

#### 3.1 attention实现

In [5]:
import torch
import math
import torch.nn.functional as F
import torch.nn as nn

def attention(query,key,value,mask=None,dropout=None):
    '''
    query,key,value size: [batch,seq_len,d_k]
    mask: [batch_size,seq_len,seq_len]
    return:
        context: [batch,seq_len,d_k]
        attention_weight : [batch,seq_len,seq_len]
    '''
    d_k = query.size(-1)
    # [batch,seq_len,seq_len]
    scores = torch.matmul(query,key.transpose(-2,-1))/math.sqrt(d_k)
    if mask is not None:
        scores.masked_fill(mask==0,-1e9)
    p_attn = F.softmax(scores,dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    # [batch,seq_len,d_k]
    context_vec = torch.matmul(p_attn,value)
    return context_vec,p_attn

#### 3.2 MultiHead attention实现

$$MultiHead(Q,K,V) = Concat(head_1,head_2,...,head_h)W^O$$

$$head_i = Attention(QW_i^Q,KW_i^K,VW_i^V)$$

In [6]:
# multi head attention
'''
MultiHead(Q,K,V) = Concat(head1,...headh)W^O
head_i

'''
class MultiHeadedAttention(nn.Module):
    def __init__(self,h,d_model,dropout_rate = 0.1):
        super(MultiHeadedAttention,self).__init__()
        assert d_model % h == 0
        self.d_k = d_model / h
        self.h = h
        self.query_linear = nn.Linear(d_model,d_model)
        self.key_linear = nn.Linear(d_model,d_model)
        self.value_linear = nn.Linear(d_model,d_model)
        self.proj_linear = nn.Linear(d_model,d_model)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout_rate)
        
    def forward(self,query,key,value,mask=None):
        '''
        mask : [batch_size,seq_len,seq_len]
        '''
        batch_size = query.size(0)
        query = self.query_linear(query)
        key = self.key_linear(key)
        value = self.value_linear(value)
        
        # split by heads
        # [batch_sizes * heads,seq_len,d_k]
        query = query.view(batch_size * self.h,-1,self.d_k)
        key = key.view(batch_size * self.h,-1,self.d_k)
        value = value.view(batch_size * self.h,-1,self.d_k)
        
        if mask is not None:
            # [batch_size * h,seq_len,seq_len]
            mask = mask.repeat(self.h,1,1)
        
        # context [batch_size * h,seq_len,d_k]
        context,self.attn = attention(query,key,value,mask,self.dropout)
        context = context.contiguous().view(batch_size,-1,self.d_k * self.h)
        return self.porj_linear(context)

#### 3.3 Layer normalization的实现

In [7]:
class LayerNorm(nn.Module):
    def __init__(self,features,eps=1e-6):
        super(LayerNorm,self).__init__()
        self.alpha = nn.Parameter(torch.ones(features))
        self.beta = nn.Parameter(torch.zeros(features))
        self.eps = eps
    
    def forward(self,x):
        mean = x.mean(-1,keepdim=True)
        std = x.std(-1,keepdim=True)
        return self.alpha * (x-mean)/(std + self.eps) + self.beta

#### 3.4 Position-wise Feed-Forward Networks的实现

In [8]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self,d_model,d_ff,dropout_rate = 0.1):
        super(PositionwiseFeedForward,self).__init__()
        self.w_1 = nn.Linear(d_model,d_ff)
        self.w_2 = nn.Linear(d_ff,d_model)
        self.dropout = nn.Dropout(dropout_rate)
    def forward(self,x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

#### 3.5 Embedding and softmax的实现

In [ ]:
class Embeddings(nn.Module):
    def __init__(self,vocab_size,d_model):
        super(Embeddings,self).__init__()
        self.word_embedding = nn.Embedding(vocab_size,d_model)
        self.d_model = d_model
        
    def forward(self,x):
        return self.word_embedding(x) * math.sqrt(self.d_model)

#### 3.6 Positional Encoding的实现

- 偶数列 dim i:
$$PE(pos,i) = sin(pos/10000^{i/d_{model}})$$

- 奇数列 dim i:
$$PE(pos,i) = cos(pos/10000^{(i-1)/d_{model}})$$ 

In [14]:
from torch.autograd import Variable
import numpy as np
class PositionalEncoding(nn.Module):
    def __init__(self,d_model,dropout_rate=0.1,max_len = 500):
        super(PositionalEncoding,self).__init__()
        self.dropout = nn.Dropout(p=dropout_rate)
        
        # [max_len,d_model]
        positional_encoding = np.array([
            [pos/np.pow(10000,2.0 * (j//2)/d_model) for j in range(d_model)]
        ] for pos in range(max_len))
        # 偶数列使用sin,奇数列使用cos
        positional_encoding[:,0::2] = np.sin(positional_encoding[:,0::2])
        positional_encoding[:,1::2] = np.cos(positional_encoding[:,1::2])
        
        # [1,max_len,seq_len]
        pe = torch.from_numpy(positional_encoding).unsqueeze(0)
        # 在内存中定义一个常量，同时，模型保存和加载的时候可以写入和读出。
        self.register_buffer('pe',pe)
    def forward(self,x):
        x = x + Variable(self.pe[:,:x.size(1)],requires_grad=False)
        return self.dropout(x)

#### 3.7 masking

In [1]:
# masking shi

# padding mask 
def padding_mask(seq_query,seq_key):
    '''
    function: padding用到三个地方，encoder,decoder,encoder-decoder
    input:
        seq_query: [batch_size,query_len]
        seq_key: [batch_size,key_len]
    return : [batch_size,query_len,key_len]
    '''
    query_len = seq_query.size(1)
    # pad is 0
    pad_mask = seq_k.eq(0)
    # [batch,seq_len,key_len]
    pad_mask = pad_mask.unsqueeze(1).expand(-1, len_q, -1)
    return pad_mask

# sequence mask
# 产生一个上三角矩阵，上三角的值全为1，下三角的值全为0，对角线也是0
def sequence_mask(seq):
    '''
    mask future info
    input: seq: [batch,seq_len]
    return mask : [batch_size,seq_len,seq_len]
    '''
    batch_size,seq_len = seq.size()
    mask = torch.triu(torch.ones((seq_len,seq_len),dtype=torch.unit8),diagonal=1)
    mask = mask.unsqueeze(0).expand(batch_size,1,1)
    return mask

#### 3.8 SublayerConnection

&emsp;&emsp;the output of each sub-layer is LayerNorm(x + sublayer(x)),where sublayer(x) is the function implemented by the sub-layer itself,we apply dropout to the output of each sub-layer,before it is added to the sub-layer inpuyt and normalized

&emsp;&emsp;output = LayerNorm(x + dropout(subLayer(x)))

In [15]:
class SublayerConnection(nn.Module):
    def __init__(self,size,dropout_rate = 0.1):
        super(SublayerConnection,self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self,x,sublayer):
        return self.norm(x + self.dropout(sublayer(x)))

### 4. 其他细节和参考文献

&emsp;&emsp;目前Transformer 大致的细节都以代码的形式实现了，还有mask部分没有实现，接下来在代码详细写一下，然后跑一下程序。

**参考文献**

[Transformer模型笔记](https://zhuanlan.zhihu.com/p/39034683)

[The Annotated Transformer](http://nlp.seas.harvard.edu/2018/04/03/attention.html)

[Transformer的PyTorch实现](https://blog.csdn.net/stupid_3/article/details/83184691)

[Transformer解读（论文 + PyTorch源码）](https://blog.csdn.net/Magical_Bubble/article/details/89083225#7_Tricks_72)

[Transform详解(超详细) Attention is all you need论文](https://zhuanlan.zhihu.com/p/63191028)

[碎碎念：Transformer的细枝末节](https://zhuanlan.zhihu.com/p/60821628)